In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt, freqz
from scipy.stats import pearsonr
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../../..')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import get_binary_stimuli, get_sample_interval, load, save, to_string

## Load Neural Data

In [3]:
session = 'Mary-Anesthesia-20160809-01'
# session = 'MrJones-Anesthesia-20160113-01.mat'
# filename = fr'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolPuffTone/{session}.mat'
filename = fr'/om/user/eisenaj/ChaoticConsciousness/data/propofolPuffTone/{session}.mat'
print("Loading data ...")
start = time.process_time()
electrode_info, lfp, lfp_schema, session_info, spike_times, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'unitInfo'], verbose=False)
spike_times = spike_times[0]
dt = lfp_schema['smpInterval'][0]
T = lfp.shape[0]

# f = h5py.File(filename, 'r')
# airPuff_binary, audio_binary = get_binary_stimuli(f)

print(f"Data loaded (took {time.process_time() - start:.2f} seconds)")

Loading data ...
Data loaded (took 153.67 seconds)


# Correlations

In [ ]:
t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)
data_path = f"/om/user/eisenaj/ChaoticConsciousness/results/propofol/correlations"
os.makedirs(data_path, exist_ok=True)

PCA_dim = -1
end_step = T - int(30/dt) # take off 30 seconds

# end_step = 15000
# for window in [2.5]:
for window in [0.15, 0.25, 0.5, 1, 2.5, 5]:
# for window in [0.25, 0.5, 1, 2.5, 5]:
#     stride = window
    stride = 10
    run_data = {}
    
    # ================
    # RUN PARAMS
    # ================
    run_data['run_params'] = dict(
        window=window,
        stride=stride,
        PCA_dim=PCA_dim,
        end_step=end_step,
        unit_indices={}
    )
    
    run_data['results'] = {}
    
    # ================
    # ALL AREAS
    # ================
    
    print(f"window = {window}, all areas")
    areas = np.unique(electrode_info['area'])
    unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]
    data = lfp[:end_step, unit_indices]
    
    results = {}
    
    num_windows = int(end_step/int(stride/dt))
    results['correlations'] = np.zeros((num_windows, len(unit_indices), len(unit_indices)))
    results['p_vals'] = np.zeros((num_windows, len(unit_indices), len(unit_indices)))
    for i in tqdm(range(num_windows)):
        window_data = data[int(stride/dt)*i:int(stride/dt)*i + int(window/dt)]
        for j, unit1 in enumerate(unit_indices):
            for k, unit2 in enumerate(unit_indices):
                results['correlations'][i, j, k], results['p_vals'][i, j, k] = pearsonr(window_data[:, unit1], window_data[:, unit2])
    
    
    run_data['results']['all'] = results
    run_data['run_params']['unit_indices']['all'] = unit_indices

    save(run_data, os.path.join(data_path, f"VAR_{os.path.basename(filename)[:-4]}_window_{window}_stride_{stride}_{timestamp}"))

window = 0.15, all areas


  0%|          | 0/705 [00:00<?, ?it/s]

window = 0.25, all areas


  0%|          | 0/705 [00:00<?, ?it/s]

window = 0.5, all areas


  0%|          | 0/705 [00:00<?, ?it/s]

window = 1, all areas


  0%|          | 0/705 [00:00<?, ?it/s]